In [10]:
# Creating Spark Context
from pyspark import SparkContext
sc = SparkContext("local", "Sentimentanalysis")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/28 17:18:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
!pip install textblob

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.3/626.3 KB 8.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 37.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 KB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.5/782.5 KB 42.3 MB/s eta 0:00:00


In [11]:
from textblob import TextBlob
from datetime import datetime


In [12]:
data = [
    ("CaptainGiggles", datetime(2024, 11, 28, 10, 0, 0).isoformat(), "Great product, very useful!"),
    ("SirLaughsALot", datetime(2024, 11, 27, 15, 30, 0).isoformat(), "Average quality, could be better."),
    ("PrincessSnickers", datetime(2024, 11, 26, 18, 45, 0).isoformat(), "Loved it! Highly recommended."),
    ("CountChuckle", datetime(2024, 11, 25, 14, 20, 0).isoformat(), "Not worth the price."),
    ("BaronGigglepants", datetime(2024, 11, 24, 11, 10, 0).isoformat(), "Excellent performance and durability."),
    ("LadyGuffaw", datetime(2024, 11, 23, 19, 5, 0).isoformat(), "Decent product for the price."),
    ("DukeOfHah", datetime(2024, 11, 22, 13, 50, 0).isoformat(), "The product arrived damaged."),
    ("MadameMirth", datetime(2024, 11, 21, 16, 30, 0).isoformat(), "Great design, but lacks features."),
    ("CountessChuckles", datetime(2024, 11, 20, 20, 40, 0).isoformat(), "Exceeded my expectations."),
    ("SirSmirks", datetime(2024, 11, 19, 9, 25, 0).isoformat(), "Terrible experience. Would not buy again."),
]

# Create an RDD from the data
rdd = sc.parallelize(data)

In [15]:
for element in rdd.collect():
    print(element)

('CaptainGiggles', '2024-11-28T10:00:00', 'Great product, very useful!')
('SirLaughsALot', '2024-11-27T15:30:00', 'Average quality, could be better.')
('PrincessSnickers', '2024-11-26T18:45:00', 'Loved it! Highly recommended.')
('CountChuckle', '2024-11-25T14:20:00', 'Not worth the price.')
('BaronGigglepants', '2024-11-24T11:10:00', 'Excellent performance and durability.')
('LadyGuffaw', '2024-11-23T19:05:00', 'Decent product for the price.')
('DukeOfHah', '2024-11-22T13:50:00', 'The product arrived damaged.')
('MadameMirth', '2024-11-21T16:30:00', 'Great design, but lacks features.')
('CountessChuckles', '2024-11-20T20:40:00', 'Exceeded my expectations.')
('SirSmirks', '2024-11-19T09:25:00', 'Terrible experience. Would not buy again.')


In [33]:
def calculate_sentiment(row):
    polarity = TextBlob(row[2]).sentiment.polarity
    sentiment = "neutral"
    if polarity < 0:
        sentiment = "negative"
    elif polarity > 0:
        sentiment = "positive"   
    return (row[0], row[1], row[2], sentiment)


In [34]:
rdd2 = rdd.map(calculate_sentiment)

In [35]:
for element in rdd2.collect():
    print(element)

('CaptainGiggles', '2024-11-28T10:00:00', 'Great product, very useful!', 'positive')
('SirLaughsALot', '2024-11-27T15:30:00', 'Average quality, could be better.', 'positive')
('PrincessSnickers', '2024-11-26T18:45:00', 'Loved it! Highly recommended.', 'positive')
('CountChuckle', '2024-11-25T14:20:00', 'Not worth the price.', 'negative')
('BaronGigglepants', '2024-11-24T11:10:00', 'Excellent performance and durability.', 'positive')
('LadyGuffaw', '2024-11-23T19:05:00', 'Decent product for the price.', 'positive')
('DukeOfHah', '2024-11-22T13:50:00', 'The product arrived damaged.', 'neutral')
('MadameMirth', '2024-11-21T16:30:00', 'Great design, but lacks features.', 'positive')
('CountessChuckles', '2024-11-20T20:40:00', 'Exceeded my expectations.', 'neutral')
('SirSmirks', '2024-11-19T09:25:00', 'Terrible experience. Would not buy again.', 'negative')


In [39]:
rdd3 = rdd2.filter(lambda x : "neutral" in x[3])

In [40]:
for element in rdd3.collect():
    print(element)

('DukeOfHah', '2024-11-22T13:50:00', 'The product arrived damaged.', 'neutral')
('CountessChuckles', '2024-11-20T20:40:00', 'Exceeded my expectations.', 'neutral')
